## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-20-11-47-32 +0000,startribune,Live: Target’s Brian Cornell to hand CEO job t...,https://www.startribune.com/target-ceo-brian-c...
1,2025-08-20-11-32-32 +0000,nypost,RFK Jr. comes clean on his unusual workout att...,https://nypost.com/2025/08/20/us-news/rfk-jr-c...
2,2025-08-20-11-32-11 +0000,bbc,Man sets fire to check-in area at Milan's Malp...,https://www.bbc.com/news/articles/ce934lgp3lmo...
3,2025-08-20-11-32-00 +0000,wsj,Target Bets a Lifelong Employee Can Steer Its ...,https://www.wsj.com/business/retail/target-new...
4,2025-08-20-11-31-57 +0000,nyt,Preparations for a Move on Gaza City Have Star...,https://www.nytimes.com/2025/08/20/world/middl...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2353/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
55,trump,49
471,ukraine,24
266,russia,15
273,deal,15
144,will,14


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
326,2025-08-19-14-07-00 +0000,wsj,"After months of effort, President Trump says a...",https://www.wsj.com/world/trump-plans-to-get-p...,126
167,2025-08-19-22-54-43 +0000,nypost,Russia launches largest strike on Ukraine in w...,https://nypost.com/2025/08/19/world-news/russi...,125
216,2025-08-19-20-30-23 +0000,latimes,Contributor: Trump's Russia and Ukraine summit...,https://www.latimes.com/opinion/story/2025-08-...,118
50,2025-08-20-09-35-05 +0000,bbc,Kremlin plays down Zelensky talks as Trump war...,https://www.bbc.com/news/articles/cn92e52rpjxo...,102
259,2025-08-19-18-30-03 +0000,nypost,Russia open to bilateral or trilateral talks w...,https://nypost.com/2025/08/19/us-news/russia-o...,95


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
326,126,2025-08-19-14-07-00 +0000,wsj,"After months of effort, President Trump says a...",https://www.wsj.com/world/trump-plans-to-get-p...
313,56,2025-08-19-14-54-58 +0000,nyt,Netanyahu Faces Pressure From Far Right Over N...,https://www.nytimes.com/2025/08/19/world/middl...
140,56,2025-08-20-01-00-00 +0000,wsj,Putin Calls Zelensky the West’s Illegitimate P...,https://www.wsj.com/world/putin-zelensky-meeti...
271,48,2025-08-19-18-03-18 +0000,latimes,SoCal's worst heat wave this year: How long wi...,https://www.latimes.com/california/story/2025-...
269,43,2025-08-19-18-16-27 +0000,nyt,Why Were Air Canada Flight Attendants Striking...,https://www.nytimes.com/2025/08/19/business/ai...
206,40,2025-08-19-20-49-17 +0000,nypost,UK free speech crackdown sees up to 30 people ...,https://nypost.com/2025/08/19/world-news/uk-fr...
228,39,2025-08-19-19-55-47 +0000,nypost,Texas Dem Rep. Nicole Collier slept in state H...,https://nypost.com/2025/08/19/us-news/texas-de...
9,36,2025-08-20-11-08-31 +0000,nypost,"Professor warned Bryan Kohberger would stalk, ...",https://nypost.com/2025/08/20/us-news/professo...
12,35,2025-08-20-11-06-00 +0000,wsj,Billionaire Bill Ackman has a new fascination:...,https://www.wsj.com/us-news/education/bill-ack...
202,34,2025-08-19-21-04-20 +0000,nypost,Pro-Adams PAC calling NYC voters to rip ‘sore ...,https://nypost.com/2025/08/19/us-news/pro-adam...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
